# 1. Importamos los datos

Importar paquetes y leer los datos.

In [1]:
import pandas as pd
import numpy as np

df = pd.read_excel('/Users/pablo/Desktop/Drive pablo.fdzalb/6o curso/TFG economía/Datos/Indices_vivienda_distritosAMB.xlsx', sheet_name = 'Distritos')
df

,CPRO,LITPRO,CUMUN,LITMUN,CUDIS,BI_ALVHEPCO_TVC_15,BI_ALVHEPCO_TVU_15,ALQM2mes_LV_M_VC_15,ALQM2mes_LV_25_VC_15,ALQM2mes_LV_75_VC_15,...,pop_density,industrial_waste,agriculture_workers_rate,industry_workers_rate,construction_workers_rate,services_workers_rate,population_growth,new_houses,places_of_interest,mob_estud
0,8,Barcelona,8015,Badalona,801501,424.0,115.0,7.381939,5.724355,9.333333,...,10529.1,15748.85,0.000354,0.102235,0.113163,0.784248,-0.130000,3.2,0.216460,0.891922
1,8,Barcelona,8015,Badalona,801502,690.0,89.0,7.267442,5.793288,8.620690,...,10529.1,15748.85,0.000354,0.102235,0.113163,0.784248,0.054602,3.2,0.216460,0.891922
2,8,Barcelona,8015,Badalona,801503,695.0,110.0,7.739814,6.288410,9.655414,...,10529.1,15748.85,0.000354,0.102235,0.113163,0.784248,0.151405,3.2,0.216460,0.891922
3,8,Barcelona,8015,Badalona,801504,727.0,67.0,7.575758,6.378369,8.698705,...,10529.1,15748.85,0.000354,0.102235,0.113163,0.784248,0.022071,3.2,0.216460,0.891922
4,8,Barcelona,8015,Badalona,801505,426.0,79.0,7.558140,6.274086,9.180327,...,10529.1,15748.85,0.000354,0.102235,0.113163,0.784248,0.100403,3.2,0.216460,0.891922
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,8,Barcelona,8245,Santa Coloma de Gramenet,824506,657.0,13.0,7.265396,6.126530,8.333333,...,17041.3,2348.38,0.000359,0.054475,0.152709,0.792457,0.242798,0.3,0.109715,0.855187
122,8,Barcelona,8282,Tiana,828201,205.0,144.0,7.890675,6.561957,9.388853,...,1133.2,3352.41,0.007458,0.079748,0.090075,0.822719,0.124611,13.4,0.110803,0.531646
123,8,Barcelona,8289,Torrelles de Llobregat,828901,120.0,109.0,6.576923,5.084746,8.148438,...,451.8,92.40,0.006591,0.081921,0.155367,0.756121,0.081449,0.3,NaN,0.591304
124,8,Barcelona,8301,Viladecans,830101,901.0,79.0,7.352941,6.076345,8.677083,...,3270.0,4535.18,0.002259,0.131495,0.112635,0.753612,0.045445,1.6,0.164646,0.850964


# 2. Limpieza de datos: exclusión BCN y eliminación de variables que no nos interesan / variables con multicolinearidad perfecta o no perfecta

### a. Exclusión de distritos de BCN y eliminación de variables que no nos interesan

In [2]:
# Ahora voy a hacer también otro DataFrame en el que excluiré los distritos de BCN, 
# que no nos interesa para los modelos. Al nuevo DataFrame lo llamaré df_noBCN.
df_noBCN = df.drop(labels = df[df['LITMUN'] == 'Barcelona'].index, axis = 0)

# Ahora eliminaré las variables que no nos interesan por motivos temporales (los datos del MITMA para antes del 2021).
## La primera columna para datos del 2021 es BI_ALVHEPCO_TVC_21. ¿Cuál es su índice?
#position = df_noBCN.columns.get_loc('BI_ALVHEPCO_TVC_21')
#print(position)

# Como el índice de la primera columna de interés es 125, tenemos que eliminar 
# las columnas anteriores, a excepción de las primeras 5 columnas (que dan datos identificativos, columnas 0 a 4).
columns_to_drop = list(range(5, 125))
df_noBCN = df_noBCN.drop(df_noBCN.columns[columns_to_drop], axis = 1)

# Ahora eliminaré el resto de variables del MITMA que no nos interesan.
df_noBCN = df_noBCN.drop(labels = ['ALQM2mes_LV_25_VC_21', 'ALQM2mes_LV_75_VC_21',
                                    'ALQM2mes_LV_25_VU_21', 'ALQM2mes_LV_75_VU_21', 
                                    'ALQTBID12_M_VC_21', 'ALQTBID12_25_VC_21',
                                    'ALQTBID12_75_VC_21', 'ALQTBID12_M_VU_21', 
                                    'ALQTBID12_25_VU_21', 'ALQTBID12_75_VU_21',
                                    'SLVM2_M_VC_21', 'SLVM2_25_VC_21', 'SLVM2_75_VC_21',
                                    'SLVM2_M_VU_21', 'SLVM2_25_VU_21', 'SLVM2_75_VU_21',
                                    'ALQM2mes_LV_M_M_21'],
                                    axis = 1)

### b. Eliminación de variables para las que hay multicolinearidad perfecta (+ sector económico, que hemos descartado porque no nos encajan)

In [3]:
# Ahora excluiré las variables en las que hay multicolinearidad perfecta (o casi),
# y que hemos decidido descartar.
df_noBCN = df_noBCN.drop(labels = ['houses', 'households', 'owned_houses', 
                                   'immigrants_rate', 'now_studying_superior_rate', 
                                   'agriculture_workers_rate', 'industry_workers_rate',
                                   'construction_workers_rate', 'services_workers_rate', 
                                   'avg_inc_pers', 'rented_houses_ine'], axis = 1)

### c. Eliminación de las variables para las que hay multicolinearidad no perfecta

In [4]:
df_noBCN = df_noBCN.drop(labels = ['now_studying_univ_rate'], axis = 1)

### d. Eliminación de variables con demasiados missing values y demás que no interesan

In [5]:
df_noBCN.isna().sum()
# Hay missing values también en la variable places_of_interest, por lo que la voy
# a eliminar de los datos.

CPRO                      0
LITPRO                    0
CUMUN                     0
LITMUN                    0
CUDIS                     0
BI_ALVHEPCO_TVC_21        0
BI_ALVHEPCO_TVU_21        0
ALQM2mes_LV_M_VC_21       2
ALQM2mes_LV_M_VU_21      26
time_pub_trans            0
time_priv_trans           0
str_line_dist             0
has_beach                 0
has_parks                 0
parks_surface             0
pt_lines                  0
avg_inc_hh                0
ibi_urb                   0
ibi_rur                   0
cat_update                0
vpo                       0
rented_houses_mitma       0
population                0
foreign_born_rate         0
superior_studies_rate     0
unemployment_rate         0
has_tram                  0
has_bus                   0
has_metro                 0
has_rodalies              0
has_fgc                   0
pop_density               0
industrial_waste          0
population_growth         0
new_houses                0
places_of_interest  

Solo hay missing values para 3 variables: las variables dependientes y places_of_interest.

Además, si se analiza para cada tipo de vivienda se puede ver que desgraciadamente los missing values de places_of_interest casi no coinciden con los de las variables dependientes.

Por otro lado, *has_bus* tiene un valor de 1 para todas las observaciones, por lo que no aporta ningún tipo de información a la regresión.

In [6]:
df_noBCN = df_noBCN.drop(labels = ['has_bus', 'places_of_interest'], axis = 1)

# 3. Creación de datasets para viviendas colectivas y unifamiliares - a nivel de distrito
Esta sección es independiente del resto.

### a. Viviendas colectivas

Eliminación de filas con missing values para la columna de precios de viviendas colectivas, y creación de un DataFrame que utilizaremos para el análisis de los precios de las viviendas colectivas.

In [7]:
df_noBCN_VC = df_noBCN.drop(labels = ['ALQM2mes_LV_M_VU_21', 'BI_ALVHEPCO_TVC_21', 'BI_ALVHEPCO_TVU_21'], axis = 1)
df_noBCN_VC = df_noBCN_VC.dropna(subset = ['ALQM2mes_LV_M_VC_21']).reset_index(drop = True)

In [8]:
df_noBCN_VC
# Nos quedamos con 114 filas (126 - 10 de BCN - 2 con missing values en la variable dependiente)

,CPRO,LITPRO,CUMUN,LITMUN,CUDIS,ALQM2mes_LV_M_VC_21,time_pub_trans,time_priv_trans,str_line_dist,has_beach,...,unemployment_rate,has_tram,has_metro,has_rodalies,has_fgc,pop_density,industrial_waste,population_growth,new_houses,mob_estud
0,8,Barcelona,8015,Badalona,801501,9.510514,27,40,9.63,1,...,0.081458,0,1,1,0,10529.1,15748.85,-0.130000,3.2,0.891922
1,8,Barcelona,8015,Badalona,801502,9.259259,34,35,9.55,0,...,0.139621,0,1,1,0,10529.1,15748.85,0.054602,3.2,0.891922
2,8,Barcelona,8015,Badalona,801503,10.512800,28,35,8.43,1,...,0.092935,1,1,0,0,10529.1,15748.85,0.151405,3.2,0.891922
3,8,Barcelona,8015,Badalona,801504,9.331875,41,40,10.18,0,...,0.114213,0,0,0,0,10529.1,15748.85,0.022071,3.2,0.891922
4,8,Barcelona,8015,Badalona,801505,9.523810,33,40,10.16,1,...,0.087060,0,1,1,0,10529.1,15748.85,0.100403,3.2,0.891922
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,8,Barcelona,8245,Santa Coloma de Gramenet,824506,8.841814,29,35,8.03,0,...,0.190450,0,1,0,0,17041.3,2348.38,0.242798,0.3,0.855187
110,8,Barcelona,8282,Tiana,828201,9.935488,44,40,13.01,0,...,0.073831,0,0,0,0,1133.2,3352.41,0.124611,13.4,0.531646
111,8,Barcelona,8289,Torrelles de Llobregat,828901,8.593750,57,55,16.43,0,...,0.084045,0,0,0,0,451.8,92.40,0.081449,0.3,0.591304
112,8,Barcelona,8301,Viladecans,830101,9.368233,36,40,14.69,1,...,0.115896,0,0,1,0,3270.0,4535.18,0.045445,1.6,0.850964


In [9]:
df_noBCN_VC.columns

Index(['CPRO', 'LITPRO', 'CUMUN', 'LITMUN', 'CUDIS', 'ALQM2mes_LV_M_VC_21',
       'time_pub_trans', 'time_priv_trans', 'str_line_dist', 'has_beach',
       'has_parks', 'parks_surface', 'pt_lines', 'avg_inc_hh', 'ibi_urb',
       'ibi_rur', 'cat_update', 'vpo', 'rented_houses_mitma', 'population',
       'foreign_born_rate', 'superior_studies_rate', 'unemployment_rate',
       'has_tram', 'has_metro', 'has_rodalies', 'has_fgc', 'pop_density',
       'industrial_waste', 'population_growth', 'new_houses', 'mob_estud'],
      dtype='object')

In [10]:
df_noBCN_VC.isna().sum()

CPRO                     0
LITPRO                   0
CUMUN                    0
LITMUN                   0
CUDIS                    0
ALQM2mes_LV_M_VC_21      0
time_pub_trans           0
time_priv_trans          0
str_line_dist            0
has_beach                0
has_parks                0
parks_surface            0
pt_lines                 0
avg_inc_hh               0
ibi_urb                  0
ibi_rur                  0
cat_update               0
vpo                      0
rented_houses_mitma      0
population               0
foreign_born_rate        0
superior_studies_rate    0
unemployment_rate        0
has_tram                 0
has_metro                0
has_rodalies             0
has_fgc                  0
pop_density              0
industrial_waste         0
population_growth        0
new_houses               0
mob_estud                0
dtype: int64

Guardo los datos en un .csv y en un .dta (para Stata).

In [11]:
df_noBCN_VC.to_csv('df_noBCN_VC.csv')
df_noBCN_VC.to_stata('df_noBCN_VC.dta')

### b. Viviendas unifamiliares

In [12]:
df_noBCN_VU = df_noBCN.drop(labels = ['ALQM2mes_LV_M_VC_21', 'BI_ALVHEPCO_TVC_21', 'BI_ALVHEPCO_TVU_21'], axis = 1)
df_noBCN_VU = df_noBCN_VU.dropna(subset = ['ALQM2mes_LV_M_VU_21']).reset_index(drop = True)

In [13]:
df_noBCN_VU
# Nos quedamos con 90 filas (126 - 10 de BCN - 26 con missing values en la variable dependiente)

,CPRO,LITPRO,CUMUN,LITMUN,CUDIS,ALQM2mes_LV_M_VU_21,time_pub_trans,time_priv_trans,str_line_dist,has_beach,...,unemployment_rate,has_tram,has_metro,has_rodalies,has_fgc,pop_density,industrial_waste,population_growth,new_houses,mob_estud
0,8,Barcelona,8015,Badalona,801501,6.336940,27,40,9.63,1,...,0.081458,0,1,1,0,10529.1,15748.85,-0.130000,3.2,0.891922
1,8,Barcelona,8015,Badalona,801502,5.102041,34,35,9.55,0,...,0.139621,0,1,1,0,10529.1,15748.85,0.054602,3.2,0.891922
2,8,Barcelona,8015,Badalona,801503,4.153728,28,35,8.43,1,...,0.092935,1,1,0,0,10529.1,15748.85,0.151405,3.2,0.891922
3,8,Barcelona,8015,Badalona,801504,7.817114,41,40,10.18,0,...,0.114213,0,0,0,0,10529.1,15748.85,0.022071,3.2,0.891922
4,8,Barcelona,8015,Badalona,801505,5.223990,33,40,10.16,1,...,0.087060,0,1,1,0,10529.1,15748.85,0.100403,3.2,0.891922
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,8,Barcelona,8245,Santa Coloma de Gramenet,824506,5.691869,29,35,8.03,0,...,0.190450,0,1,0,0,17041.3,2348.38,0.242798,0.3,0.855187
86,8,Barcelona,8282,Tiana,828201,7.389119,44,40,13.01,0,...,0.073831,0,0,0,0,1133.2,3352.41,0.124611,13.4,0.531646
87,8,Barcelona,8289,Torrelles de Llobregat,828901,6.646586,57,55,16.43,0,...,0.084045,0,0,0,0,451.8,92.40,0.081449,0.3,0.591304
88,8,Barcelona,8301,Viladecans,830101,5.357430,36,40,14.69,1,...,0.115896,0,0,1,0,3270.0,4535.18,0.045445,1.6,0.850964


In [14]:
df_noBCN_VU.columns

Index(['CPRO', 'LITPRO', 'CUMUN', 'LITMUN', 'CUDIS', 'ALQM2mes_LV_M_VU_21',
       'time_pub_trans', 'time_priv_trans', 'str_line_dist', 'has_beach',
       'has_parks', 'parks_surface', 'pt_lines', 'avg_inc_hh', 'ibi_urb',
       'ibi_rur', 'cat_update', 'vpo', 'rented_houses_mitma', 'population',
       'foreign_born_rate', 'superior_studies_rate', 'unemployment_rate',
       'has_tram', 'has_metro', 'has_rodalies', 'has_fgc', 'pop_density',
       'industrial_waste', 'population_growth', 'new_houses', 'mob_estud'],
      dtype='object')

In [15]:
df_noBCN_VU.isna().sum()

CPRO                     0
LITPRO                   0
CUMUN                    0
LITMUN                   0
CUDIS                    0
ALQM2mes_LV_M_VU_21      0
time_pub_trans           0
time_priv_trans          0
str_line_dist            0
has_beach                0
has_parks                0
parks_surface            0
pt_lines                 0
avg_inc_hh               0
ibi_urb                  0
ibi_rur                  0
cat_update               0
vpo                      0
rented_houses_mitma      0
population               0
foreign_born_rate        0
superior_studies_rate    0
unemployment_rate        0
has_tram                 0
has_metro                0
has_rodalies             0
has_fgc                  0
pop_density              0
industrial_waste         0
population_growth        0
new_houses               0
mob_estud                0
dtype: int64

Guardo los datos en un .csv y en un .dta (para Stata).

In [16]:
df_noBCN_VU.to_csv('df_noBCN_VU.csv')
df_noBCN_VU.to_stata('df_noBCN_VU.dta')

# 4. Creación de datasets para viviendas colectivas y unifamiliares - a nivel de municipio
Esta sección es independiente del resto. La sección de "Viviendas colectivas" también es independiente de la sección de "Viviendas unifamiliares" (pero no al revés).

El punto de partida es el DataFrame del final de la sección 2. A continuación, iré agregando los datos por grupos de variables en un DataFrame.

### **Viviendas colectivas**

#### i. Variable dependiente

Aquí haré las medias ponderadas de las medianas para cada municipio. Primero voy a eliminar los missing values de cada variable dependiente (aquellas observaciones para las que hay <= 10 viviendas del tipo en cuestión por distrito) para poder realizar las medias ponderadas sin que haya errores.

In [17]:
df_noBCN_nona_VC = df_noBCN.dropna(subset = ['ALQM2mes_LV_M_VC_21']).reset_index(drop = True)

Ahora voy a calcular la media de las medianas de cada distrito para cada municipio y cada tipo de viviendas. Para calcular las medias, voy a utilizar una media ponderada, obtenida a partir del número de viviendas de cada tipo para cada distrito (en la variable "BI_ALVHEPCO_TVC_21" y "BI_ALVHEPCO_TVU_21").

In [18]:
## Cálculo de las medias ponderadas para las viviendas colectivas. Agruparé por 
## CUMUN (código del municipio) y LITMUN (nombre del municipio) para hacer los cálculos.

df_noBCN_VC_municip = df_noBCN_nona_VC.groupby(['CUMUN', 'LITMUN']).agg({'ALQM2mes_LV_M_VC_21': lambda x: sum(x * df_noBCN_nona_VC.loc[x.index, 'BI_ALVHEPCO_TVC_21']) / sum(df_noBCN_nona_VC.loc[x.index, 'BI_ALVHEPCO_TVC_21'])}).reset_index()

df_noBCN_VC_municip

,CUMUN,LITMUN,ALQM2mes_LV_M_VC_21
0,8015,Badalona,9.221991
1,8020,Begues,9.283657
2,8054,Castellbisbal,7.576446
3,8056,Castelldefels,11.561019
4,8068,Cervelló,7.885955
5,8072,Corbera de Llobregat,7.281553
6,8073,Cornellà de Llobregat,10.235450
7,8077,Esplugues de Llobregat,11.524223
8,8089,Gavà,9.792644
9,8101,"Hospitalet de Llobregat, L'",11.620557


He comprobado manualmente para Gavá si la media ponderada de la mediana está bien (9.79 € / m2), y efectivamente es así. Además, también he mirado la mediana de los datos por municipio en la base de datos del MITMA, y parece que la media ponderada que he calculado es un buen estimador de la mediana de precio / m2 por municipio.

Ahora iré juntando los nuevos datos agrupados a este DataFrame inicial de *df_noBCN_VC_municip*.

#### ii. Variables de desplazamiento

Para las variables *time_pub_trans*, *time_priv_trans* y *str_line_dist* voy a calcular las medias por municipio ponderadas con la población, de manera que el tiempo de desplazamiento sea lo más representativo posible de un ciudadano en cada municipio. Para ello, utilizaré el mismo cálculo que para las variable dependiente, pero ahora ponderaré por *population*.

Uniré los resultados al DataFrame *df_noBCN_VC_municip*, con un inner merge.

In [19]:
# time_pub_trans, ponderado con population
df_noBCN_VC_municip = pd.merge(df_noBCN_VC_municip, df_noBCN_nona_VC.groupby(['CUMUN', 'LITMUN']).agg({'time_pub_trans': lambda x: sum(x * df_noBCN_nona_VC.loc[x.index, 'population']) / sum(df_noBCN_nona_VC.loc[x.index, 'population'])}).reset_index())

# time_priv_trans, ponderado con population
df_noBCN_VC_municip = pd.merge(df_noBCN_VC_municip, df_noBCN_nona_VC.groupby(['CUMUN', 'LITMUN']).agg({'time_priv_trans': lambda x: sum(x * df_noBCN_nona_VC.loc[x.index, 'population']) / sum(df_noBCN_nona_VC.loc[x.index, 'population'])}).reset_index())

# str_line_dist, ponderado con population
df_noBCN_VC_municip = pd.merge(df_noBCN_VC_municip, df_noBCN_nona_VC.groupby(['CUMUN', 'LITMUN']).agg({'str_line_dist': lambda x: sum(x * df_noBCN_nona_VC.loc[x.index, 'population']) / sum(df_noBCN_nona_VC.loc[x.index, 'population'])}).reset_index())

Ahora uniré las dummies de desplazamiento. El criterio será el siguiente: si hay algún 1 en alguno de los distritos, entonces en la observación para el municipio habrá un 1; en caso contrario, tendrá un 0.

In [20]:
# has_tram
df_noBCN_VC_municip = pd.merge(df_noBCN_VC_municip, df_noBCN_nona_VC.groupby(['CUMUN', 'LITMUN']).has_tram.apply(lambda x: 1 if np.sum(x >= 1) else 0).reset_index())

# has_metro
df_noBCN_VC_municip = pd.merge(df_noBCN_VC_municip, df_noBCN_nona_VC.groupby(['CUMUN', 'LITMUN']).has_metro.apply(lambda x: 1 if np.sum(x >= 1) else 0).reset_index())

# has_rodalies
df_noBCN_VC_municip = pd.merge(df_noBCN_VC_municip, df_noBCN_nona_VC.groupby(['CUMUN', 'LITMUN']).has_rodalies.apply(lambda x: 1 if np.sum(x >= 1) else 0).reset_index())

# has_fgc
df_noBCN_VC_municip = pd.merge(df_noBCN_VC_municip, df_noBCN_nona_VC.groupby(['CUMUN', 'LITMUN']).has_fgc.apply(lambda x: 1 if np.sum(x >= 1) else 0).reset_index())

Ahora, falta la variable *pt_lines*. En este caso, como la variable toma los mismos valores para los distintos distritos de un municipio, si hago la media me dará el mismo valor por municipio.

In [21]:
df_noBCN_VC_municip = pd.merge(df_noBCN_VC_municip, df_noBCN_nona_VC.groupby(['CUMUN', 'LITMUN']).pt_lines.mean().reset_index())

#### iii. Variables socioeconómicas

Para la variable *population*, simplemente haré la suma de la población por municipio de entre todos los distritos.

In [22]:
df_noBCN_VC_municip = pd.merge(df_noBCN_VC_municip, df_noBCN_nona_VC.groupby(['CUMUN', 'LITMUN']).population.sum().reset_index())

Para las variables *unemployment_rate*, *foreign_born_rate*, *superior_studies_rate*, *avg_inc_hh* y *population_growth* voy a calcular las medias por municipio ponderadas por la población (*population*). 

In [23]:
# unemployment_rate, ponderado con population
df_noBCN_VC_municip = pd.merge(df_noBCN_VC_municip, df_noBCN_nona_VC.groupby(['CUMUN', 'LITMUN']).agg({'unemployment_rate': lambda x: sum(x * df_noBCN_nona_VC.loc[x.index, 'population']) / sum(df_noBCN_nona_VC.loc[x.index, 'population'])}).reset_index())

# foreign_born_rate, ponderado con population
df_noBCN_VC_municip = pd.merge(df_noBCN_VC_municip, df_noBCN_nona_VC.groupby(['CUMUN', 'LITMUN']).agg({'foreign_born_rate': lambda x: sum(x * df_noBCN_nona_VC.loc[x.index, 'population']) / sum(df_noBCN_nona_VC.loc[x.index, 'population'])}).reset_index())

# superior_studies_rate, ponderado con population
df_noBCN_VC_municip = pd.merge(df_noBCN_VC_municip, df_noBCN_nona_VC.groupby(['CUMUN', 'LITMUN']).agg({'superior_studies_rate': lambda x: sum(x * df_noBCN_nona_VC.loc[x.index, 'population']) / sum(df_noBCN_nona_VC.loc[x.index, 'population'])}).reset_index())

# avg_inc_hh, ponderado con population
df_noBCN_VC_municip = pd.merge(df_noBCN_VC_municip, df_noBCN_nona_VC.groupby(['CUMUN', 'LITMUN']).agg({'avg_inc_hh': lambda x: sum(x * df_noBCN_nona_VC.loc[x.index, 'population']) / sum(df_noBCN_nona_VC.loc[x.index, 'population'])}).reset_index())

# population_growth, ponderado con population
df_noBCN_VC_municip = pd.merge(df_noBCN_VC_municip, df_noBCN_nona_VC.groupby(['CUMUN', 'LITMUN']).agg({'population_growth': lambda x: sum(x * df_noBCN_nona_VC.loc[x.index, 'population']) / sum(df_noBCN_nona_VC.loc[x.index, 'population'])}).reset_index())

Para las variables *pop_density*, *ibi_urb*, *ibi_rur*, *cat_update* y *mob_estud*, como están ya por municipio, he hecho la media para agregar los datos.

In [24]:
df_noBCN_VC_municip = pd.merge(df_noBCN_VC_municip, df_noBCN_nona_VC.groupby(['CUMUN', 'LITMUN']).pop_density.mean().reset_index())

df_noBCN_VC_municip = pd.merge(df_noBCN_VC_municip, df_noBCN_nona_VC.groupby(['CUMUN', 'LITMUN']).ibi_urb.mean().reset_index())

df_noBCN_VC_municip = pd.merge(df_noBCN_VC_municip, df_noBCN_nona_VC.groupby(['CUMUN', 'LITMUN']).ibi_rur.mean().reset_index())

df_noBCN_VC_municip = pd.merge(df_noBCN_VC_municip, df_noBCN_nona_VC.groupby(['CUMUN', 'LITMUN']).cat_update.mean().reset_index())

df_noBCN_VC_municip = pd.merge(df_noBCN_VC_municip, df_noBCN_nona_VC.groupby(['CUMUN', 'LITMUN']).mob_estud.mean().reset_index())

#### iv. Real estate characteristics

Para la variable *has_parks*, el criterio será el siguiente: si hay algún 1 en alguno de los distritos, entonces en la observación para el municipio habrá un 1; en caso contrario, tendrá un 0.

In [25]:
# has_parks
df_noBCN_VC_municip = pd.merge(df_noBCN_VC_municip, df_noBCN_nona_VC.groupby(['CUMUN', 'LITMUN']).has_parks.apply(lambda x: 1 if np.sum(x >= 1) else 0).reset_index())

Para la variable *rented_houses_mitma*, simplemente voy a sumar las viviendas totales disponibles en alquiler por municipio.

In [26]:
df_noBCN_VC_municip = pd.merge(df_noBCN_VC_municip, df_noBCN_nona_VC.groupby(['CUMUN', 'LITMUN']).rented_houses_mitma.sum().reset_index())

Para las variables *industrial_waste*, *vpo* y *new_houses*, como están ya por municipio, he hecho la media para agregar los datos.

In [27]:
df_noBCN_VC_municip = pd.merge(df_noBCN_VC_municip, df_noBCN_nona_VC.groupby(['CUMUN', 'LITMUN']).industrial_waste.mean().reset_index())

df_noBCN_VC_municip = pd.merge(df_noBCN_VC_municip, df_noBCN_nona_VC.groupby(['CUMUN', 'LITMUN']).vpo.mean().reset_index())

df_noBCN_VC_municip = pd.merge(df_noBCN_VC_municip, df_noBCN_nona_VC.groupby(['CUMUN', 'LITMUN']).new_houses.mean().reset_index())

Para la variable *parks_surface* voy a calcular las medias por municipio ponderadas por la población (*population*), porque se supone que será más representativa la superficie de un parque que tiene cerca mucha gente.

In [28]:
# parks_surface, ponderado con population
df_noBCN_VC_municip = pd.merge(df_noBCN_VC_municip, df_noBCN_nona_VC.groupby(['CUMUN', 'LITMUN']).agg({'parks_surface': lambda x: sum(x * df_noBCN_nona_VC.loc[x.index, 'population']) / sum(df_noBCN_nona_VC.loc[x.index, 'population'])}).reset_index())

#### v. Otras

Para la variable *has_beach*, el criterio será el siguiente: si hay algún 1 en alguno de los distritos, entonces en la observación para el municipio habrá un 1 (tiene playa); en caso contrario, tendrá un 0.

In [29]:
# has_beach
df_noBCN_VC_municip = pd.merge(df_noBCN_VC_municip, df_noBCN_nona_VC.groupby(['CUMUN', 'LITMUN']).has_beach.apply(lambda x: 1 if np.sum(x >= 1) else 0).reset_index())

### *Resultado*

In [30]:
df_noBCN_VC_municip

,CUMUN,LITMUN,ALQM2mes_LV_M_VC_21,time_pub_trans,time_priv_trans,str_line_dist,has_tram,has_metro,has_rodalies,has_fgc,...,ibi_rur,cat_update,mob_estud,has_parks,rented_houses_mitma,industrial_waste,vpo,new_houses,parks_surface,has_beach
0,8015,Badalona,9.221991,33.162462,34.941060,8.611524,1,1,1,0,...,0.7990,1998.0,0.891922,1,11168,15748.85,22.0,3.2,56811.224947,1
1,8020,Begues,9.283657,75.000000,55.000000,21.630000,0,0,0,0,...,0.7100,2010.0,0.726937,1,378,224.76,0.0,1.9,10237.000000,0
2,8054,Castellbisbal,7.576446,73.000000,60.000000,18.710000,0,0,0,0,...,0.5150,2005.0,0.797131,1,594,493377.58,17.0,1.9,14998.750000,0
3,8056,Castelldefels,11.561019,46.296728,41.330646,20.590545,0,0,1,0,...,0.4200,2006.0,0.805291,1,5221,311.89,7.0,0.6,43860.000000,1
4,8068,Cervelló,7.885955,55.000000,55.000000,17.980000,0,0,0,0,...,1.0000,2013.0,0.580128,1,427,3222.28,0.0,2.3,7718.000000,0
5,8072,Corbera de Llobregat,7.281553,86.000000,60.000000,20.800000,0,0,0,0,...,0.7700,1999.0,0.609259,1,857,477.83,0.0,0.5,3624.000000,0
6,8073,Cornellà de Llobregat,10.235450,25.058716,33.468550,8.342974,1,1,1,1,...,0.4660,2006.0,0.776492,1,5618,21819.17,0.0,3.7,73506.216694,0
7,8077,Esplugues de Llobregat,11.524223,29.889485,31.967658,6.847914,1,1,1,0,...,1.0100,1997.0,0.718048,1,2653,7434.97,0.0,0.1,62683.705262,0
8,8089,Gavà,9.792644,47.820494,40.000000,16.754339,0,0,1,0,...,0.9000,2001.0,0.871301,1,2578,15742.80,30.0,1.7,59651.250000,1
9,8101,"Hospitalet de Llobregat, L'",11.620557,22.381607,27.585133,5.318290,1,1,1,1,...,0.4300,2000.0,0.813806,1,18815,28295.74,39.0,2.9,65532.909292,0


### *Guardo los datos*

In [31]:
df_noBCN_VC_municip.to_csv('df_noBCN_VC_municip.csv')
df_noBCN_VC_municip.to_stata('df_noBCN_VC_municip.dta')

### **Viviendas unifamiliares**
Aquí he hecho un merge con el DataFrame que he creado para las viviendas colectivas, ya que nos sirven los mismos datos agregados. Lo único que haré será cambiar la variable dependiente. 

Obviamente, por eso, he cambiado los siguientes nombres de DataFrames: ***df_noBCN_nona_VU*** en vez de *df_noBCN_nona_VC* y ***df_noBCN_VU_municip*** en vez de *df_noBCN_VC_municip*.

In [32]:
df_noBCN_nona_VU = df_noBCN.dropna(subset = ['ALQM2mes_LV_M_VU_21']).reset_index(drop = True)

In [33]:
# Creo un DataFrame nuevo con los datos agregados para la mediana de precio / m2 para VCs
df_noBCN_VU_municip = df_noBCN_nona_VU.groupby(['CUMUN', 'LITMUN']).agg({'ALQM2mes_LV_M_VU_21': lambda x: sum(x * df_noBCN_nona_VU.loc[x.index, 'BI_ALVHEPCO_TVU_21']) / sum(df_noBCN_nona_VU.loc[x.index, 'BI_ALVHEPCO_TVU_21'])}).reset_index()

# Hago un outer merge con el DataFrame de VCs por municipio
df_noBCN_VU_municip = pd.merge(df_noBCN_VU_municip, df_noBCN_VC_municip, how = 'outer')

# Elimino las filas con valores NaN de nuevo, que se han vuelto a incluir por el outer merge (solo 1, Badia del Vallès)
df_noBCN_VU_municip = df_noBCN_VU_municip.dropna(subset = ['ALQM2mes_LV_M_VU_21']).reset_index(drop = True)

# Elimino la columna de precios de VCs, que aquí no interesan
df_noBCN_VU_municip = df_noBCN_VU_municip.drop(labels = ['ALQM2mes_LV_M_VC_21'], axis = 1)

### *Resultado*

In [34]:
df_noBCN_VU_municip

,CUMUN,LITMUN,ALQM2mes_LV_M_VU_21,time_pub_trans,time_priv_trans,str_line_dist,has_tram,has_metro,has_rodalies,has_fgc,...,ibi_rur,cat_update,mob_estud,has_parks,rented_houses_mitma,industrial_waste,vpo,new_houses,parks_surface,has_beach
0,8015,Badalona,5.372441,33.162462,34.941060,8.611524,1,1,1,0,...,0.7990,1998.0,0.891922,1,11168,15748.85,22.0,3.2,56811.224947,1
1,8020,Begues,7.241620,75.000000,55.000000,21.630000,0,0,0,0,...,0.7100,2010.0,0.726937,1,378,224.76,0.0,1.9,10237.000000,0
2,8054,Castellbisbal,6.819341,73.000000,60.000000,18.710000,0,0,0,0,...,0.5150,2005.0,0.797131,1,594,493377.58,17.0,1.9,14998.750000,0
3,8056,Castelldefels,11.941936,46.296728,41.330646,20.590545,0,0,1,0,...,0.4200,2006.0,0.805291,1,5221,311.89,7.0,0.6,43860.000000,1
4,8068,Cervelló,6.607143,55.000000,55.000000,17.980000,0,0,0,0,...,1.0000,2013.0,0.580128,1,427,3222.28,0.0,2.3,7718.000000,0
5,8072,Corbera de Llobregat,6.207310,86.000000,60.000000,20.800000,0,0,0,0,...,0.7700,1999.0,0.609259,1,857,477.83,0.0,0.5,3624.000000,0
6,8073,Cornellà de Llobregat,8.118825,25.058716,33.468550,8.342974,1,1,1,1,...,0.4660,2006.0,0.776492,1,5618,21819.17,0.0,3.7,73506.216694,0
7,8077,Esplugues de Llobregat,12.535144,29.889485,31.967658,6.847914,1,1,1,0,...,1.0100,1997.0,0.718048,1,2653,7434.97,0.0,0.1,62683.705262,0
8,8089,Gavà,8.252999,47.820494,40.000000,16.754339,0,0,1,0,...,0.9000,2001.0,0.871301,1,2578,15742.80,30.0,1.7,59651.250000,1
9,8101,"Hospitalet de Llobregat, L'",6.273387,22.381607,27.585133,5.318290,1,1,1,1,...,0.4300,2000.0,0.813806,1,18815,28295.74,39.0,2.9,65532.909292,0


### *Guardo los datos*

In [35]:
df_noBCN_VU_municip.to_csv('df_noBCN_VU_municip.csv')
df_noBCN_VU_municip.to_stata('df_noBCN_VU_municip.dta')